In [43]:
import pandas as pd
import numpy as np

In [459]:
class Wind:
    """ relative winddirection for now as input from 0-180 on the boat looking in the heading direction
    H = [m] height of the vessel m
    B = [m] width of the vessel 
    L = [m] length of the vessel
    rel_winddir = [deg] relative winddirection from 0-180 from looking at winddirection and heading ! manually
    loaded_depth = [m] depth of vessel loaded
    unloaded_depth = [m] depth of vessel unloaded
    load = 'loaded'/'unloaded', for depth and cd
    V_g = [m/s] speed over ground 
    heading = [deg] direction 0 degrees is north , 45 degrees is northeast. 
    winddirection = [deg] geographic wind direction
    U_wind = [m/s] windspeed
    rho_air = [kg/m3] density of air
    """
    def __init__(
        #""" relative winddirection for now as input from 0-180 on the boat looking in the heading direction
        #"""
        self,
        B, 
        L,
        H ,
        rel_winddir,
        loaded_depth,
        unloaded_depth,
        load = 'loaded',
     
        V_g = 0.3,
        heading = 40,# degrees (NE)
        winddirection = 60,# degrees (not relative but real)
        U_wind = 2,
        rho_air = 1.225
        
        
        
    ):
        self.B = B
        self.L = L
        self.H = H
        self.load = load
        self.V_g = V_g
        self.heading = heading
        self.winddirection = winddirection
        self.U_wind = U_wind
        self.rho_air = rho_air
        self.rel_winddir = rel_winddir
        self.loaded_depth = loaded_depth
        self.unloaded_depth = unloaded_depth
       
    
    def calculate_rel_winddir (self):
        #phi_w_ref = angle of attack  
        #(https://sciendo.com/pdf/10.2478/ntpe-2018-0013)
        # pffff still not found the right relations for angle, winddirection. 
            
        self.phi_w_ref = self.winddirection - self.heading  
        return self.phi_w_ref
    
    def calculate_C_d(self):
        # for a tanker ( ITTC: ) 
        # cda = -cx (cx given for ships)
        
        cd=[[0,0.86,0.96],[10,0.76,0.93],[20,0.62,0.85],
        [30,0.45,0.73],[40,0.32,0.62],[50,0.21,0.47],
        [60,0.13,0.34],[70,0.06,0.17],[80,0.04,0.06],
        [90,-0.02,-0.05],[100,-0.08,-0.14],[110,-0.19,-0.22],
        [120,-0.29,-0.29],[130,-0.38,-0.40],[140,-0.47,-0.53],
        [150,-0.56,-0.66],[160,-0.61,-0.75],[170,-0.66,-0.79],
        [180,-0.63,-0.77]]
    
        df=pd.DataFrame(cd,columns=['Relative wind direction','loaded','unloaded'])

        if self.load == 'loaded':
            self.C_d = df['loaded'].where(df['Relative wind direction'] == self.rel_winddir).fillna(0).values[1]
    
        elif self.load == 'unloaded':
            self.C_d = df['unloaded'].where(df['Relative wind direction'] == self.rel_winddir).fillna(0).values[1]
    
        return self.C_d
    
    def calculate_Cd_0(self):
        if self.load == 'loaded':
            self.Cd_0 = 0.86                            
        elif self.load == 'unloaded':
            self.Cd_0 = 0.96
        return self.Cd_0 
    
    def calculate_H_above_water(self):
        if self.load == 'loaded':
            self.H_above = self.H - self.loaded_depth 
        elif self.load == 'unloaded':
            self.H_above = self.H - self.unloaded_depth
        return self.H_above
    
    def calculate_A_xv(self):
        #cubed barge assumed 
        angle=0
        if self.rel_winddir <91:
            self.A_xv = self.calculate_H_above_water() * (self.B * np.cos(self.rel_winddir*np.pi/180) + self.L * np.sin(self.rel_winddir*np.pi/180))
        elif self.rel_winddir > 90:
            angle = 90-(self.rel_winddir -90)
            self.A_xv = self.calculate_H_above_water() * (self.B * np.cos(angle) + self.L * np.sin(angle))
        return self.A_xv , angle
    
    def calculate_wind_resistance(self):
        self.R_wind = 0.5 * self.calculate_C_d() * self.rho_air * self.calculate_A_xv()[0] * self.U_wind **2 - 0.5 * self.rho_air * self.calculate_Cd_0() * self.calculate_A_xv()[0] * self.V_g **2

                    
#     todo: 1) add the angle between wind force direction and vessel sailing direction to determine wind force to the vessel 2) add wind speed as input on the edge 
# drag coefficient of angle 0 should be in the minus part. 
# Raa = 0.5 * rho_a * C_da(phiWref) * A_xv * V_wrref**2 - 0.5 * rho_a * C_da(0) * A_xv * Vg**2
# V_g komt later pas erin.
#loaded/unloaded
# negative value is in the Cd  value. so negative winddirection gives negative value because of cd. 
    
        return self.R_wind

In [460]:
p =  Wind(11,60,rel_winddir=45,load = 'loaded',V_g = 0.3,heading = 40,winddirection = 60,U_wind = 15,rho_air = 1.225, H = 4, loaded_depth = 3.45, unloaded_depth = 2)

In [461]:
p.calculate_rel_winddir()

20

In [462]:
print(p.calculate_H_above_water(),'H_above')
print(p.calculate_A_xv(), 'Axv')
print(p.calculate_C_d(), 'C_d')
print(p.calculate_rel_winddir(), 'rel_winddir')

0.5499999999999998 H_above
(27.61251980533467, 0) Axv
0.0 C_d
20 rel_winddir


In [463]:
p.calculate_wind_resistance()

-1.3090405326714034